<a href="https://colab.research.google.com/github/rami-nava/AlgoritmoGeneticoAulas/blob/main/AlgoritmoGeneticoAulas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuracion Inicial de Variables

Seteamos los parametros de entrada para utilizar en el AG que creemos de forma manual, esto nos permite setear variables como los metodos de SELECCCION, CRUZAMIENTO, CANTIDAD DE GENERACIONES, etc ..

In [ ]:
class Config:
    def __init__(self):
        self.POPULATION_SIZE = 100
        self.GENERATIONAL_LEAP = 0.7
        self.NUMBER_OF_GENERATIONS = 200
        self.MUTATION_PROBABILITY = 0.1
        # CROSSOVER_FUNCTION VALUES:
        #   "RANDOM": cruza binomial aleatoria
        #   "SINGLE_POINT": cruza simple
        #   "MASK": cruza binomial con máscara doble
        RANDOM = "RANDOM"
        SINGLE_POINT = "SINGLE_POINT"
        MASK = "MASK"
        self.CROSSOVER_FUNCTION = RANDOM
        # MASKS
        self.MASK_FIRST_CHILD = "XXYYXYXXXYYYXYXYYYXXXYXYYXXYYYYXXXYYXYXX"
        self.MASK_SECOND_CHILD = "YYXXYXYYYXXXYXYXXXYYXYXYYXXXYYXYYXXYXXYX"
        #SELECTION FUNCTION
        TOURNAMENT = "TOURNAMENT"
        ROULETTE = "ROULETTE"
        self.SELECTION_FUNCTION = TOURNAMENT


CONFIG = Config()

# Declaracion de Individuos

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/IA/TP AG

Mounted at /content/drive/
/content/drive/My Drive/IA/TP AG


In [ ]:
from enum import Enum

# Días válidos para dictado de clases
class Dia(Enum):
    LUNES = 0
    MARTES = 1
    MIERCOLES = 2
    JUEVES = 3
    VIERNES = 4

# Representa una asignación tentativa de un curso
class Gen:
    def __init__(self, id_curso: int, id_aula: int, dias: list[Dia]):
        self.id_curso = id_curso
        self.id_aula = id_aula
        self.dias = dias

    def print(self):
      print(f"id curso:{self.id_curso}"),
      print(f"id aula:{self.id_aula}"),
      #print(f"dias:{self.dias}"),
      diasAsignados = []
      for dia in self.dias:
        diasAsignados.append(dia.name)
      print(f"dias: {diasAsignados}")


# Datos relevantes de un curso
class Curso:
    def __init__(self, id_curso, cant_dias: int, cant_alumnos: int,
                 requiere_proyector: bool = False,
                 requiere_pizarron: bool = False,
                 requiere_bancos_indiv: bool = False,
                 dias_preferidos: list[Dia] = []):
        self.id_curso = id_curso
        self.cant_dias = cant_dias
        self.cant_alumnos = cant_alumnos
        self.requiere_proyector = requiere_proyector
        self.requiere_pizarron = requiere_pizarron
        self.requiere_bancos_indiv = requiere_bancos_indiv
        self.dias_preferidos = dias_preferidos

# Datos relevantes de un aula
class Aula:
    def __init__(self, id_aula, capacidad: int, tiene_proyector: bool = False,
                 tiene_pizarron: bool = False, bancos_indiv: bool = False):
        self.id_aula = id_aula
        self.capacidad = capacidad
        self.tiene_proyector = tiene_proyector
        self.tiene_pizarron = tiene_pizarron
        self.bancos_indiv = bancos_indiv

In [ ]:
# Carga de los cursos y aulas
import pandas as pd

# Convierte un string tipo "LUNES;MIERCOLES" a [Dia.LUNES, Dia.MIERCOLES]
def parse_dias(dias_str):
    return [Dia[dia.strip()] for dia in dias_str.split(";")]

def cargar_cursos(path="cursos.csv"):
    df = pd.read_csv(path)
    return [
        Curso(
            id_curso = i,
            cant_dias=row["cant_dias"],
            cant_alumnos=row["cant_alumnos"],
            requiere_proyector=bool(row["requiere_proyector"]),
            requiere_pizarron=bool(row["requiere_pizarron"]),
            requiere_bancos_indiv=bool(row["requiere_bancos_indiv"]),
            dias_preferidos=parse_dias(row["dias_preferidos"])
        )
        for i, row in df.iterrows()
    ]

def cargar_aulas(path="aulas.csv"):
    df = pd.read_csv(path)
    return [
        Aula(
            id_aula = i,
            capacidad=row["capacidad"],
            tiene_proyector=bool(row["tiene_proyector"]),
            tiene_pizarron=bool(row["tiene_pizarron"]),
            bancos_indiv=bool(row["bancos_indiv"])
        )
        for i, row in df.iterrows()
    ]

In [ ]:
#Esto nos permite ver la info de nuestra lista de forma TABULADA
!pip install tabulate

In [ ]:
#Importamos la lib que acabamos de instalar para poder ver la informacion
from tabulate import tabulate

cursos = cargar_cursos()
aulas = cargar_aulas()

#Convertimos la informacion que tenemos de los cursos en una tabla, y de las aulas en otra
table_cursos = [[c.cant_dias, c.cant_alumnos, c.requiere_proyector, c.requiere_pizarron, c.requiere_bancos_indiv, [d.name for d in c.dias_preferidos]] for c in cursos]
table_aulas = [[a.capacidad, a.tiene_proyector, a.tiene_pizarron, a.bancos_indiv] for a in aulas]

#Para poder visualizar la tabla de forma correcta, utilizamos la funcion TABULATE
print(tabulate(table_cursos, headers=["Cantidad de días", "Cantidad de alumnos", "Requiere proyector", "Requiere pizarrón", "Requiere bancos individuales", "Días preferidos"], tablefmt="fancy_grid"))
print(tabulate(table_aulas, headers=["Capacidad", "Tiene proyectos", "Tiene pizarrón","Tiene bancos individuales"], tablefmt="fancy_grid"))

╒════════════════════╤═══════════════════════╤══════════════════════╤═════════════════════╤════════════════════════════════╤══════════════════════════╕
│   Cantidad de días │   Cantidad de alumnos │ Requiere proyector   │ Requiere pizarrón   │ Requiere bancos individuales   │ Días preferidos          │
╞════════════════════╪═══════════════════════╪══════════════════════╪═════════════════════╪════════════════════════════════╪══════════════════════════╡
│                  1 │                    35 │ True                 │ False               │ False                          │ ['LUNES']                │
├────────────────────┼───────────────────────┼──────────────────────┼─────────────────────┼────────────────────────────────┼──────────────────────────┤
│                  1 │                    25 │ False                │ True                │ True                           │ ['MARTES']               │
├────────────────────┼───────────────────────┼──────────────────────┼───────────────────

# Definicion de un individuo

In [ ]:
#Primero importamos las libs que necesitaremos utilizar a lo largo de toda la ejecucion
from random import random
from random import randint
from random import choice
from random import sample
import plotly.express as px

In [ ]:
#Establecemos las penalizaciones y bonificaciones que se tomaran en cuenta para la funcion fitness
class PenalizacionesConfig:
    def __init__(self):
        #De restriccion
        self.CAPACIDAD_INSUFICIENTE = -1
        self.FALTA_PROYECTOR = -1
        self.FALTA_PIZARRON = -1
        self.BANCOS_INCORRECTOS = -1
        self.DIA_NO_PREFERIDO = -1
        #Invalidas
        self.DIAS_REPETIDOS = -20
        self.DIAS_INSUFICIENTES = -10
        self.AULA_OCUPADA = -20
PENALIZACION = PenalizacionesConfig()

#Para llevar un registro de la forma en la que se penaliza
MODOS_PENALIZACION = {
    "CAPACIDAD_INSUFICIENTE": "Penaliza por cada alumno que excede - Incremental porcentual factor/100",
    "FALTA_PROYECTOR": "Incremental porcentual factor/100",
    "FALTA_PIZARRON": "Incremental porcentual factor/100",
    "BANCOS_INCORRECTOS": "Incremental porcentual factor/100",
    "DIA_NO_PREFERIDO": "Penaliza por cada dia no asignado - InIncremental porcentual factor/100cremental",
    "DIAS_REPETIDOS": "Incremental porcentual factor/100",
    "DIAS_INSUFICIENTES": "Incremental porcentual factor/100",
    "AULA_OCUPADA": "Penaliza por cada conflicto - Incremental porcentual factor/100"
}

class BonificacionesConfig:
    def __init__(self):
        self.CAPACIDAD_SUFICIENTE = 1
        self.TIENE_PROYECTOR = 1
        self.TIENE_PIZARRON = 1
        self.BANCOS_CORRECTOS = 1
        self.DIA_PREFERIDO = 1
        ## NUEVOS
        self.DIAS_NO_REPETIDOS = 0
        self.DIAS_SUFICIENTES = 0
        self.AULA_NO_OCUPADA = 0

BONIFICACION = BonificacionesConfig()

MODOS_BONIFICACION = {
    "CAPACIDAD_SUFICIENTE": "Incremental porcentual factor/100",
    "TIENE_PROYECTOR": "Incremental porcentual factor/100",
    "TIENE_PIZARRON": "Incremental porcentual factor/100",
    "BANCOS_CORRECTOS": "Incremental porcentual factor/100",
    "DIA_PREFERIDO": "Por dia - Incremental porcentual factor/100",
    "DIAS_NO_REPETIDOS": "Incremental porcentual factor/100",
    "DIAS_SUFICIENTES": "Incremental porcentual factor/100",
}

### Penalizacion simple

In [ ]:
#Declaramos la clase Individuo, en nuestro caso 1 individuo no sera 1 curso o aula, sino una posible COMBINACION DE CURSOS Y AULAS.
#El individuo tendra la funcion de FITNESS, el cual nos va a permitir saber si es el mejor individuo a seleccionar.
class Individual:

    #El cromosoma nos dira la combinacion de aulas y cursos para tal individuo
    def __init__(self, chromosome=[]):
        if not chromosome:
            self.chromosome: Gen = []
            for curso in cursos:
                aula = randint(0,9)
                if curso.cant_dias == 2:
                    dias = [Dia(randint(0,4)),Dia(randint(0,4))]
                else:
                    dias = [Dia(randint(0,4))]
                gen = Gen(id_curso=curso.id_curso,id_aula=aula,dias=dias)
                self.chromosome.append(gen)
        else:
            self.chromosome = chromosome

    # Penalización/bonificación por cantidad de días asignados
    #def evaluar_dias_asignados(self, asignados: int, requeridos: int) -> int:
     #  return 1 if asignados == requeridos else -3
    def evaluar_dias_asignados(self, gen, cursos, aulas) -> int:
      score = 0;
      score += BONIFICACION.DIAS_SUFICIENTES if len(gen.dias) == len(cursos[gen.id_curso].dias_preferidos) else PENALIZACION.DIAS_INSUFICIENTES
      score += BONIFICACION.DIAS_NO_REPETIDOS if len(gen.dias) == len(set(gen.dias)) else PENALIZACION.DIAS_REPETIDOS #Penaliza si se le asignaron 2 dias iguales para cursar

         ###### NO ESTABLECIMOS UNA BONIFICACION EXCLUSIVAMENTE EN CASO DE QUE NO SE ASIGNEN DIAS IGUALES PARA CURSAR, A) USAR LA MISMA B) CREAR NUEVA BONIFICACION

      return score

    # Evalúa compatibilidad entre aula y curso
    def evaluar_aula(self, gen, cursos, aulas) -> int:
        #print(gen)
        if gen.id_aula == 0:
            return 0  # virtual

        curso = cursos[gen.id_curso]
        aula = aulas[gen.id_aula]

        score = 0
        score += BONIFICACION.CAPACIDAD_SUFICIENTE if aula.capacidad >= curso.cant_alumnos else abs(aula.capacidad - curso.cant_alumnos)*PENALIZACION.CAPACIDAD_INSUFICIENTE
        score += BONIFICACION.TIENE_PROYECTOR if curso.requiere_proyector == aula.tiene_proyector else PENALIZACION.FALTA_PROYECTOR if curso.requiere_proyector else 0
        score += BONIFICACION.TIENE_PIZARRON if curso.requiere_pizarron == aula.tiene_pizarron else PENALIZACION.FALTA_PIZARRON if curso.requiere_pizarron else 0
        score += BONIFICACION.BANCOS_CORRECTOS if curso.requiere_bancos_indiv == aula.bancos_indiv else PENALIZACION.BANCOS_INCORRECTOS
        #Corrida 1
        #tiene_dia_preferido = any(dia in curso.dias_preferidos for dia in gen.dias)
        #score += BONIFICACION.DIA_PREFERIDO if tiene_dia_preferido else PENALIZACION.DIA_NO_PREFERIDO

        score += sum(BONIFICACION.DIA_PREFERIDO if dia in curso.dias_preferidos else PENALIZACION.DIA_NO_PREFERIDO for dia in gen.dias)

        return score


    #La funcion de FITNESS nos dira el valor que tendra tal individuo, con la finalidad de obtener el que mayor valor tenga
    def fitness(self):
        score = 0
        dias_asignados = [0] * len(cursos)
        ocupacion_aulas = set()

        for gen in self.chromosome:
            score += self.evaluar_aula(gen, cursos, aulas)
            score += self.evaluar_dias_asignados(gen, cursos, aulas)

            #Corrida 1
            #penalizar = False

            for dia in gen.dias:
                key = (gen.id_aula, dia)
                if gen.id_aula != 0 and key in ocupacion_aulas:
                    #Corrida 1
                    #penalizar = True
                    score += PENALIZACION.AULA_OCUPADA

                ocupacion_aulas.add(key)
            dias_asignados[gen.id_curso] += len(gen.dias)

        #Corrida 1
        #if penalizar:
        #    score += PENALIZACION.AULA_OCUPADA


        return score

    def print(self):
      for gen in self.chromosome:
        gen.print()

### Penalizacion incremental

In [ ]:
class Individual:

    def __init__(self, chromosome=[]):
        if not chromosome:
            self.chromosome: Gen = []
            for curso in cursos:
                aula = randint(0, 9)
                if curso.cant_dias == 2:
                    dias = [Dia(randint(0, 4)), Dia(randint(0, 4))]
                else:
                    dias = [Dia(randint(0, 4))]
                gen = Gen(id_curso=curso.id_curso, id_aula=aula, dias=dias)
                self.chromosome.append(gen)
        else:
            self.chromosome = chromosome

    def calcular_penalizacion_por_tipo(self, tipo_penalizacion: str, penalizacion_base: int, contadores_penalizaciones) -> int:
        contadores_penalizaciones[tipo_penalizacion] += 1
        factor = contadores_penalizaciones[tipo_penalizacion]
        return penalizacion_base + (factor/100 * penalizacion_base)

    def calcular_bonificacion_por_tipo(self, tipo_bonificacion: str, bonificacion_base: int, contadores_bonificaciones) -> int:
        contadores_bonificaciones[tipo_bonificacion] += 1
        factor = contadores_bonificaciones[tipo_bonificacion]
        return bonificacion_base + (factor/100 * bonificacion_base)

    def evaluar_dias_asignados(self, gen, cursos, aulas, contadores_penalizaciones, contadores_bonificaciones) -> int:
        score = 0
        if len(gen.dias) == len(cursos[gen.id_curso].dias_preferidos):
            score += self.calcular_bonificacion_por_tipo("DIAS_SUFICIENTES", BONIFICACION.DIAS_SUFICIENTES, contadores_bonificaciones)
        else:
            score += self.calcular_penalizacion_por_tipo("DIAS_INSUFICIENTES", PENALIZACION.DIAS_INSUFICIENTES, contadores_penalizaciones)

        if len(gen.dias) == len(set(gen.dias)):
            score += self.calcular_bonificacion_por_tipo("DIAS_NO_REPETIDOS", BONIFICACION.DIAS_NO_REPETIDOS, contadores_bonificaciones)
        else:
            score += self.calcular_penalizacion_por_tipo("DIAS_REPETIDOS", PENALIZACION.DIAS_REPETIDOS, contadores_penalizaciones)

        return score

    def evaluar_aula(self, gen, cursos, aulas, contadores_penalizaciones, contadores_bonificaciones) -> int:
        if gen.id_aula == 0:
            return 0  # virtual

        curso = cursos[gen.id_curso]
        aula = aulas[gen.id_aula]

        score = 0
        if aula.capacidad >= curso.cant_alumnos:
            score += self.calcular_bonificacion_por_tipo("CAPACIDAD_SUFICIENTE", BONIFICACION.CAPACIDAD_SUFICIENTE, contadores_bonificaciones)
        else:
            diferencia = curso.cant_alumnos - aula.capacidad
            penalizacion_capacidad_total = diferencia * PENALIZACION.CAPACIDAD_INSUFICIENTE
            score += self.calcular_penalizacion_por_tipo("CAPACIDAD_INSUFICIENTE", penalizacion_capacidad_total, contadores_penalizaciones)

        if curso.requiere_proyector and aula.tiene_proyector:
            score += self.calcular_bonificacion_por_tipo("TIENE_PROYECTOR", BONIFICACION.TIENE_PROYECTOR, contadores_bonificaciones)
        elif curso.requiere_proyector and not aula.tiene_proyector:
            score += self.calcular_penalizacion_por_tipo("FALTA_PROYECTOR", PENALIZACION.FALTA_PROYECTOR, contadores_penalizaciones)

        if curso.requiere_pizarron and aula.tiene_pizarron:
            score += self.calcular_bonificacion_por_tipo("TIENE_PIZARRON", BONIFICACION.TIENE_PIZARRON, contadores_bonificaciones)
        elif curso.requiere_pizarron and not aula.tiene_pizarron:
            score += self.calcular_penalizacion_por_tipo("FALTA_PIZARRON", PENALIZACION.FALTA_PIZARRON, contadores_penalizaciones)

        if curso.requiere_bancos_indiv and aula.bancos_indiv:
            score += self.calcular_bonificacion_por_tipo("BANCOS_CORRECTOS", BONIFICACION.BANCOS_CORRECTOS, contadores_bonificaciones)
        elif curso.requiere_bancos_indiv and not aula.bancos_indiv:
            score += self.calcular_penalizacion_por_tipo("BANCOS_INCORRECTOS", PENALIZACION.BANCOS_INCORRECTOS, contadores_penalizaciones)

        for dia in gen.dias:
            if dia in curso.dias_preferidos:
                score += self.calcular_bonificacion_por_tipo("DIA_PREFERIDO", BONIFICACION.DIA_PREFERIDO, contadores_bonificaciones)
            else:
                score += self.calcular_penalizacion_por_tipo("DIA_NO_PREFERIDO", PENALIZACION.DIA_NO_PREFERIDO, contadores_penalizaciones)

        return score

    def fitness(self):
        # Reiniciar contadores en cada evaluación
        contadores_penalizaciones = {k: 0 for k in [
            "CAPACIDAD_INSUFICIENTE",
            "FALTA_PROYECTOR",
            "FALTA_PIZARRON",
            "BANCOS_INCORRECTOS",
            "DIA_NO_PREFERIDO",
            "DIAS_REPETIDOS",
            "DIAS_INSUFICIENTES",
            "AULA_OCUPADA"
        ]}

        contadores_bonificaciones = {k: 0 for k in [
            "CAPACIDAD_SUFICIENTE",
            "TIENE_PROYECTOR",
            "TIENE_PIZARRON",
            "BANCOS_CORRECTOS",
            "DIA_PREFERIDO",
            "DIAS_NO_REPETIDOS",
            "DIAS_SUFICIENTES",
            "AULA_NO_OCUPADA"
        ]}

        score = 0
        ocupacion_aulas = set()
        dias_asignados = [0] * len(cursos)

        for gen in self.chromosome:
            score += self.evaluar_aula(gen, cursos, aulas, contadores_penalizaciones, contadores_bonificaciones)
            score += self.evaluar_dias_asignados(gen, cursos, aulas, contadores_penalizaciones, contadores_bonificaciones)

            for dia in gen.dias:
                key = (gen.id_aula, dia)
                if gen.id_aula != 0 and key in ocupacion_aulas:
                    score += self.calcular_penalizacion_por_tipo("AULA_OCUPADA", PENALIZACION.AULA_OCUPADA, contadores_penalizaciones)
                elif gen.id_aula != 0:
                    score += self.calcular_bonificacion_por_tipo("AULA_NO_OCUPADA", BONIFICACION.AULA_NO_OCUPADA, contadores_bonificaciones)
                ocupacion_aulas.add(key)

            dias_asignados[gen.id_curso] += len(gen.dias)

        #if score < 0:
        #    score = 0

        return score

    def print(self):
        for gen in self.chromosome:
            gen.print()

In [ ]:
ind = Individual()
ind.print()

id curso:0
id aula:8
dias: ['JUEVES']
id curso:1
id aula:4
dias: ['MARTES']
id curso:2
id aula:2
dias: ['MARTES']
id curso:3
id aula:4
dias: ['LUNES']
id curso:4
id aula:0
dias: ['LUNES']
id curso:5
id aula:9
dias: ['LUNES']
id curso:6
id aula:0
dias: ['JUEVES']
id curso:7
id aula:4
dias: ['LUNES']
id curso:8
id aula:1
dias: ['JUEVES']
id curso:9
id aula:6
dias: ['LUNES']
id curso:10
id aula:9
dias: ['MIERCOLES']
id curso:11
id aula:4
dias: ['MIERCOLES']
id curso:12
id aula:2
dias: ['JUEVES']
id curso:13
id aula:1
dias: ['MARTES']
id curso:14
id aula:3
dias: ['MARTES']
id curso:15
id aula:5
dias: ['JUEVES']
id curso:16
id aula:9
dias: ['MIERCOLES']
id curso:17
id aula:6
dias: ['MIERCOLES']
id curso:18
id aula:0
dias: ['VIERNES']
id curso:19
id aula:8
dias: ['JUEVES']
id curso:20
id aula:8
dias: ['LUNES', 'MIERCOLES']
id curso:21
id aula:5
dias: ['LUNES', 'LUNES']
id curso:22
id aula:6
dias: ['MARTES', 'LUNES']
id curso:23
id aula:1
dias: ['VIERNES', 'MIERCOLES']
id curso:24
id aula:5
d

#Generacion de la Poblacion Inicial

Para empezar la problematica, generaremos la poblacion inicial a utilizar

In [ ]:
def get_initial_population():
    _population = []
    for i in range(CONFIG.POPULATION_SIZE):
        _population.append(Individual())
    return _population

Ademas, la idea es ver cual es el mejor individuo de la poblacion inicial, que luego nos permitira ver a futuro si aplicando la solucion propuesta, encontramos uno mejor que este inicial

In [ ]:
import copy
def get_best_individual(_population):
    best_ind = copy.deepcopy(_population[0])
    best_fitness = best_ind.fitness()
    for ind in _population:
        current_fitness = ind.fitness()
        if current_fitness > best_fitness:
            best_fitness = current_fitness
            best_ind = copy.deepcopy(ind)
    return best_ind

#Etapa de Seleccion

Primero en base a la configuracion que tengamos, seleccionamos el metodo de SELECCION a utilizar, en nuestro caso tendremos en consideracion los metodos TORNEO y RULETA

In [ ]:
def get_population_after_selection(old_population):
    match CONFIG.SELECTION_FUNCTION:
        case "TOURNAMENT":
            return select_population_by_tournament(old_population)
        case _:
            return select_population_by_roulette(old_population)

In [ ]:
def select_population_by_tournament(old_population):
    tournament_size = int(len(old_population) / 10)
    new_population = []
    if CONFIG.POPULATION_SIZE % 2 != 0:
        raise Exception("Cannot perform tournament if population size is not an even number")
    while len(new_population) < len (old_population):
      #realizamos distintos torneos de 1/10 elementos sobre el total de la poblacion,
      #el ganador de cada uno forma parte de la siguiente poblacion
      tournament_population = []
      for individual_index in range(tournament_size):
        tournament_population.append(choice(old_population))
      new_population.append(select_individual_by_tournament(tournament_population))
    return new_population

def select_individual_by_tournament(_population):
    if len(_population) == 1:
        return _population[0]
    else:
        new_population = []
        already_selected = []
        for _i in range(len(_population) // 2):
            individual1 = get_random_individual_not_already_selected(_population, already_selected)
            individual2 = get_random_individual_not_already_selected(_population, already_selected)
            new_population.append(competition_result(individual1, individual2))
        return select_individual_by_tournament(new_population)

def get_random_individual_not_already_selected(_population, already_selected_individual_indexes):
    _index = randint(0, len(_population) - 1)
    while _index in already_selected_individual_indexes:
        _index = randint(0, len(_population) - 1)
    already_selected_individual_indexes.append(_index)
    return _population[_index]

def competition_result(individual1, individual2):
    if individual1.fitness() >= individual2.fitness():
        return individual1
    else:
        return individual2

In [ ]:
def select_population_by_roulette(old_population):
    new_population = []
    fitness_function_sum = total_sum_fitness_function(old_population)
    for individual_index in range(len(old_population)):
        new_population.append(select_individual_by_roulette(old_population, fitness_function_sum))
    return new_population


def total_sum_fitness_function(_population):
    _sum = 0
    for ind in _population:
        _sum += ind.fitness()
    return _sum


def select_individual_by_roulette(_population, fitness_function_sum):
    _random = random() * fitness_function_sum
    _sum = 0
    _index = 0
    while _index < len(_population):
        _sum += _population[_index].fitness()
        if _sum >= _random:
            return _population[_index]
        else:
            _index += 1
    #Aca a veces no entraba en el if y terminaba devolviendo un None. Por lo que despues rompia
    return _population[len(_population)-1]

#Etapa de Cruzamiento

Primero en base a la configuracion que tengamos, seleccionamos el metodo de CRUZAMIENTO a utilizar, en nuestro caso tendremos en consideracion los metodos Cruza Binomial: Azar, Mascara Doble y Mascara Complemento

In [ ]:
def crossover(individual1, individual2):
    match CONFIG.CROSSOVER_FUNCTION:
        case "RANDOM":
            return random_crossover(individual1, individual2)
        case "MASK":
            return mask_crossover(individual1, individual2)
        case _:
            return single_point_crossover(individual1, individual2)

In [ ]:
def random_crossover(individual1, individual2):
    return [get_random_child(individual1, individual2), get_random_child(individual1, individual2)]


def get_random_child(individual1, individual2): #en este metodo los nuevos hijos obtienen su cromosoma de forma aleatoria en base a las caracteristicas de sus padres
    child_chromosome = []
    for _i in range(len(individual1.chromosome)):
        if random() <= 0.5:
            child_chromosome.append(individual1.chromosome[_i])
        else:
            child_chromosome.append(individual2.chromosome[_i])
    return Individual(child_chromosome)

In [ ]:
def mask_crossover(individual1, individual2):
    return [get_child_from_mask(individual1, individual2, CONFIG.MASK_FIRST_CHILD),
            get_child_from_mask(individual1, individual2, CONFIG.MASK_SECOND_CHILD)]


def get_child_from_mask(individual1, individual2, mask): #aca se predefine, las mascaras de los hijos, donde en base a su mascara se decide que caracteristicas hereda de su padre y cuales de su madre
    if len(individual1.chromosome) != len(mask):
        raise_invalid_mask_exception()
    child_chromosome = []
    for _i in range(len(mask)):
        if mask[_i] == 'X':
            child_chromosome.append(individual1.chromosome[_i]) #heredamos una caracteristica del padre
        elif mask[_i] == 'Y':
            child_chromosome.append(individual2.chromosome[_i]) #heredamos una caracteristica de la madre
        else:
            raise_invalid_mask_exception()
    return Individual(child_chromosome)


def raise_invalid_mask_exception():
    raise Exception("Invalid mask")

In [ ]:
def single_point_crossover(individual1, individual2):
    cutoff = round(random() * len(individual1.chromosome))
    chromosome1 = individual1.chromosome[0:cutoff] + individual2.chromosome[cutoff::] #uno hereda ciertas caracteristicas del padre y madre
    chromosome2 = individual2.chromosome[0:cutoff] + individual1.chromosome[cutoff::] #el hermano hereda lo complementario a su hermano
    child1 = Individual(chromosome1)
    child2 = Individual(chromosome2)
    return [child1, child2]

Luego de definir los distintos metodos de cruza y tener definido que metodo utilizar en el modelo, realizaremos la cruza de una determinada cantidad de individuos (definido en la CONFIGURACION INICIAL) para generar nuevos potenciales individuos, los cuales heredaran ciertas caracteristicas de sus padres en base al metodo de cruza seleccionado

In [ ]:
def get_population_after_crossover(old_population):
    gen_leap_length = (round((len(old_population) * CONFIG.GENERATIONAL_LEAP))//2) * 2 #tomaremos una determinada cantidad de padres para realizar la cruza

    #for i in range(0,len(old_population)):
      #print(i)
      #print(old_population[i])

    parents = old_population[0:gen_leap_length]
    crossed_population = []
    #print(len(parents))
    for pi in range(0, len(parents), 2): #realizaremos la cruza de a 2, padre y madre
        #print(pi)
        #print(old_population[pi])
        #print(old_population[pi+1])
        children = crossover(old_population[pi], old_population[pi + 1]) #como resultado por cada par de padres, tenemos un par de hijos
        crossed_population.append(children[0])
        crossed_population.append(children[1])
    return crossed_population + old_population[gen_leap_length::] #la poblacion final sera la de los nuevos hijos + los individuos que no tuvieron cruza

#Etapa de mutuacion

In [ ]:
def get_population_after_mutation(_population):
    mutation_probability = random()
    if mutation_probability <= CONFIG.MUTATION_PROBABILITY:
        mutation_pos = len(_population)

        while mutation_pos == len(_population):
            mutation_pos = get_mutation_position(_population)

        individual_index = mutation_pos // len(_population)
        chromosome_index = mutation_pos % len(_population[0].chromosome)

        mutar_gen(_population[individual_index].chromosome[chromosome_index])
        #if _population[individual_index].chromosome[chromosome_index] == 0:
            #_population[individual_index].chromosome[chromosome_index] = 1
        #else:
            #_population[individual_index].chromosome[chromosome_index] = 0

    return _population

def mutar_gen(gen):
    dias_disponibles = list(Dia)
    curso = cursos[gen.id_curso]

    '''aulas_validas = [i + 1 for i, a in enumerate(aulas) if a.capacidad >= curso.cant_alumnos]
    if curso.requiere_proyector:
      aulas_validas = [i for i in aulas_validas if aulas[i - 1].tiene_proyector]
    if curso.requiere_pizarron:
      aulas_validas = [i for i in aulas_validas if aulas[i - 1].tiene_pizarron]
    if curso.requiere_bancos_indiv:
      aulas_validas = [i for i in aulas_validas if aulas[i - 1].bancos_indiv]

    if aulas_validas:
      gen.id_aula = choice(aulas_validas)
    else:
      gen.id_aula = 0  # virtual como último recurso

    # Intentamos mejorar los días asignados usando días preferidos si existen
    if curso.dias_preferidos and len(curso.dias_preferidos) >= curso.cant_dias:
      gen.dias = sample(curso.dias_preferidos, k=curso.cant_dias)
    else:
      gen.dias = sample(dias_disponibles, k=curso.cant_dias)
'''

    aulas_validas = [i for i, a in enumerate(aulas) if a.capacidad >= curso.cant_alumnos]
    if curso.requiere_proyector:
      aulas_validas = [i for i in aulas_validas if aulas[i].tiene_proyector]
    if curso.requiere_pizarron:
      aulas_validas = [i for i in aulas_validas if aulas[i].tiene_pizarron]
    if curso.requiere_bancos_indiv:
      aulas_validas = [i for i in aulas_validas if aulas[i].bancos_indiv]

    #if aulas_validas:
    gen.id_aula = choice(aulas_validas)

    # Intentamos mejorar los días asignados usando días preferidos si existen
    if curso.dias_preferidos and len(curso.dias_preferidos) >= curso.cant_dias:
      gen.dias = sample(curso.dias_preferidos, k=curso.cant_dias)
    else:
      gen.dias = sample(dias_disponibles, k=curso.cant_dias)



def get_mutation_position(_population):
    return round(random() * len(_population) * len(_population[0].chromosome))

# Ejecucion de la solucion

In [ ]:
import copy

solutions = []


def chequeo_cumple(individuo):
  ocupacion_aulas = set()
  print("Evaluacion del mejor individuo")
  cant_capacidad_inco = 0
  cant_proyector_inco = 0
  cant_pizzarron_incorrecto = 0
  cant_bancos_inco = 0
  cant_diaspref_inco = 0
  cant_diaspref_inco_total = 0
  cant_ocupado =0


  for gen in individuo.chromosome:
            print(f"Aula {gen.id_aula} y curso {gen.id_curso}")
            #score += self.evaluar_aula(gen, cursos, aulas)
            curso = cursos[gen.id_curso]
            aula = aulas[gen.id_aula]

            if gen.id_aula != 0: #si no es virtual

              if aula.capacidad < curso.cant_alumnos:
                print(f"El aula no cumple con la capacidad necesaria. Capacida {aula.capacidad}, alumnos {curso.cant_alumnos}")
                cant_capacidad_inco += 1

              if curso.requiere_proyector:
                  if not aula.tiene_proyector:
                    print("Al aula le hace falta un proyector")
                    cant_proyector_inco += 1

              if curso.requiere_pizarron:
                  if aula.tiene_pizarron:
                    print("Al aula le hace falta un pizarrón")
                    cant_pizzarron_incorrecto += 1

              if curso.requiere_bancos_indiv != aula.bancos_indiv:
                  print("La asignación del tipo de bancos es incorrecta")
                  cant_bancos_inco += 1

            error_dia= False
            error_ocupado = False


            diasAsignadosCorrectamente = filter(lambda d: d in gen.dias, curso.dias_preferidos)

            for dia in curso.dias_preferidos:
              if dia not in diasAsignadosCorrectamente:
                print(f"El dia preferido {dia.name} no se le fue asignado al curso")
                cant_diaspref_inco_total +=1


            for dia in gen.dias:
                if dia not in curso.dias_preferidos:
                  print(f"El dia asignado {dia.name} no es de preferencia")
                  if not error_dia:
                    cant_diaspref_inco +=1
                    error_dia = True


                key = (gen.id_aula, dia)
                if gen.id_aula != 0 and key in ocupacion_aulas:
                    print(f"El aula {gen.id_aula} ya se encuentra ocupada para el dia {dia.name}")
                    if not error_ocupado:
                      cant_ocupado +=1
                      error_ocupado = True
                ocupacion_aulas.add(key)

            if len(gen.dias) != curso.cant_dias:
               print ("No se asignaron los dias suficientes al curso")

            print("--------------------------------------")

  total_dias_asignados = 0
  for gen in individuo.chromosome:
    total_dias_asignados += len(gen.dias)

  total_asignaciones = len(individuo.chromosome)
  print(f"Total de asignaciones {len(individuo.chromosome)}")

  print(f"Cantidad de asignaciones con capacidad incorrecta: {cant_capacidad_inco}, {cant_capacidad_inco / total_asignaciones:.2%}")
  print(f"Cantidad de asignaciones con proyector incorrecto: {cant_proyector_inco}, {cant_proyector_inco / total_asignaciones:.2%}")
  print(f"Cantidad de asignaciones con pizarrón incorrecto: {cant_pizzarron_incorrecto}, {cant_pizzarron_incorrecto / total_asignaciones:.2%}")
  print(f"Cantidad de asignaciones con tipo de bancos incorrecto: {cant_bancos_inco}, {cant_bancos_inco / total_asignaciones:.2%}")
  print(f"Cantidad de asignaciones que tienen aulas ya ocupadas: {cant_ocupado}, {cant_ocupado / total_asignaciones:.2%}")
  print(f"Cantidad de asignaciones que tienen días preferidos incorrectos: {cant_diaspref_inco}, {cant_diaspref_inco / total_asignaciones:.2%}")
  print(f"Cantidad de días preferidos no asignados: {cant_diaspref_inco_total}, {cant_diaspref_inco_total / total_dias_asignados:.2%}")

  resultado = []
  resultado.append(("capacidad incorrecta", cant_capacidad_inco, f"{cant_capacidad_inco / total_asignaciones:.2%}"))
  resultado.append(("proyector incorrecto", cant_proyector_inco, f"{cant_proyector_inco / total_asignaciones:.2%}"))
  resultado.append(("pizarrón incorrecto", cant_pizzarron_incorrecto, f"{cant_pizzarron_incorrecto / total_asignaciones:.2%}"))
  resultado.append(("tipo de bancos incorrecto", cant_bancos_inco, f"{cant_bancos_inco / total_asignaciones:.2%}"))
  resultado.append(("aulas ya ocupadas", cant_ocupado, f"{cant_ocupado / total_asignaciones:.2%}"))
  resultado.append(("días preferidos incorrectos", cant_diaspref_inco, f"{cant_diaspref_inco / total_asignaciones:.2%}"))
  resultado.append(("días preferidos no asignados", cant_diaspref_inco_total, f"{cant_diaspref_inco_total / total_dias_asignados:.2%}"))
  return resultado
  #print(f"Cantidad de asignaciones con cantidad de días incorrecta: {cant_dias_inco}, %{cant_dias_inco / total_asignaciones:.2%}")

def execute_ga_from_scratch():
    population = get_initial_population()
    best = get_best_individual(population)
    for gen_number in range(CONFIG.NUMBER_OF_GENERATIONS):
        #print("Antes de la seleccion")
        #for i in range(0,len(population)):
          #print(i)
          #print(population[i])
        population = get_population_after_selection(population)
        #print("Despues de la seleccion")
        #for i in range(0,len(population)):
          #print(i)
          #print(population[i])
        population = get_population_after_crossover(population)
        population = get_population_after_mutation(population)
        print("Best individual of generation ",
              str(gen_number + 1), " : ")
        gen_best = get_best_individual(population)
        solutions.append(gen_best.fitness())
        #gen_best.print()
        print(f"Fitness function value: {gen_best.fitness()}")
        if gen_best.fitness() > best.fitness():
            best = copy.deepcopy(gen_best)
            #lo usamos para crearnos una nueva referencia al mejor individuo y que no se modifique la referencia al mejor en las corridas


       # if gen_number % 15 == 0: #cada 15 generaciones renovamos el 10% de la poblacion
        #    cant_a_reemplazar = int(0.1 * len(population))
         #   ind_a_eliminar = sample(population, k=cant_a_reemplazar)

          #  for ind in ind_a_eliminar:
           #   population.remove(ind)

            #for i in range(cant_a_reemplazar):
             # population.append(Individual())

    return best

best_ind = execute_ga_from_scratch()
print("Mejor Individuo")
best_ind.print()
best_score = best_ind.fitness()
print(f"Mejor fitness {best_score}")

resultados = chequeo_cumple(best_ind)

graph_title = "Best score: " + str(best_score) \
              + " - Generational leap: " + str(CONFIG.GENERATIONAL_LEAP) \
              + " - Mutation probability: " + str(CONFIG.MUTATION_PROBABILITY) \
              + " - Crossover function: " + CONFIG.CROSSOVER_FUNCTION \
              + " - Selection function: " + CONFIG.SELECTION_FUNCTION

figure = px.line(x=range(0, CONFIG.NUMBER_OF_GENERATIONS), y=solutions, title=graph_title)
figure.show()

Best individual of generation  1  : 
Fitness function value: -376.53
Best individual of generation  2  : 
Fitness function value: -376.4
Best individual of generation  3  : 
Fitness function value: -269.9
Best individual of generation  4  : 
Fitness function value: -208.95000000000002
Best individual of generation  5  : 
Fitness function value: -150.89
Best individual of generation  6  : 
Fitness function value: -72.47999999999999
Best individual of generation  7  : 
Fitness function value: -71.1
Best individual of generation  8  : 
Fitness function value: -68.13999999999999
Best individual of generation  9  : 
Fitness function value: -29.590000000000003
Best individual of generation  10  : 
Fitness function value: -28.54
Best individual of generation  11  : 
Fitness function value: -8.240000000000004
Best individual of generation  12  : 
Fitness function value: 4.6800000000000015
Best individual of generation  13  : 
Fitness function value: 6.719999999999999
Best individual of generat

# Guardar resultados

In [ ]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.9 MB/s eta 0:00:00


In [ ]:
import os
import csv
from datetime import datetime
from datetime import datetime
import pytz


def guardar_config(data_dict, filename, modos=None):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        encabezado = ["Parámetro", "Valor"]
        if modos:
            encabezado.append("Modo")
        writer.writerow(encabezado)

        for key, value in data_dict.items():
            row = [key, value]
            if modos:
                row.append(modos.get(key, "desconocido"))
            writer.writerow(row)

def guardar_todas_las_configs(output_dir):
    guardar_config(CONFIG.__dict__, os.path.join(output_dir, "configuracion_general.csv"))
    guardar_config(PENALIZACION.__dict__, os.path.join(output_dir, "penalizaciones.csv"), MODOS_PENALIZACION)
    guardar_config(BONIFICACION.__dict__, os.path.join(output_dir, "bonificaciones.csv"), MODOS_BONIFICACION)

def guardar_mejor_individuo_csv(best_individual, output_dir):
    filepath = os.path.join(output_dir, "mejor_individuo.csv")
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["id_curso", "id_aula", "dias"])
        for gen in best_individual.chromosome:
            dias_str = ','.join(str(dia) for dia in gen.dias)
            writer.writerow([gen.id_curso, gen.id_aula, dias_str])

def crear_carpeta_corrida(base_dir="corridas"):
    os.makedirs(base_dir, exist_ok=True)
    local_tz = pytz.timezone('America/Argentina/Buenos_Aires')
    timestamp = datetime.now(local_tz).strftime("%d-%m-%Y _ %H%M%S")
    folder_path = os.path.join(base_dir, f"corrida _ {timestamp}")
    os.makedirs(folder_path)
    return folder_path

def guardar_metricas_individuo(resultados,output_dir):
    filepath = os.path.join(output_dir, "metricas.csv")
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Metrica", "resultado", "dias"])
        for (nombre,cantidad,porcentaje) in resultados:
          writer.writerow([nombre,cantidad,porcentaje])

In [ ]:
#Guardar configs y resultados
carpeta = crear_carpeta_corrida()

guardar_todas_las_configs(carpeta)
#figure.write_image(os.path.join(carpeta, "Grafico_fitness.png"))
guardar_mejor_individuo_csv(best_ind, carpeta)
guardar_metricas_individuo(resultados,carpeta)

print(f"Resultados guardados en: {carpeta}")

Resultados guardados en: corridas/corrida _ 01-06-2025 _ 183943
